### For tips on running notebooks in Google Colab, see:  
[pytorch on colab tips](https://pytorch.org/tutorials/beginner/colab)

In [1]:
%matplotlib inline

[Learn the Basics](intro.html) \|\| **Quickstart** \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Based on official[Quickstart Turorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)   
With a lot of additi
==========

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

Working with data
-----------------

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like CIFAR, COCO ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset.   

Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.


In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports automatic batching, sampling,
shuffling and multiprocess data loading.   
Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.


In [4]:
#Hyper parameter
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


NCHW is an acronym describing the order of the axes in a tensor containing image data samples.

    N: Number of data samples.

    C: Image channels. A red-green-blue (RGB) image will have 3 channels. 1 is a greayscale image.

    H: Image height.

    W: Image width.

------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from  [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).   
 

To accelerate operations in the neural network, we move it to
the GPU or MPS if available.


In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
input = torch.randn(32, 1, 5, 5)
# With default parameters 1, -1
m = nn.Flatten()
output = m(input)
output.size()

torch.Size([32, 25])

In [7]:
input = torch.randn(64, 1, 28, 28)  # 64 rows
# With default parameters 1, -1
m = nn.Flatten()
output = m(input)
output.size()

torch.Size([64, 784])

In [8]:
# Uncomment nn. and press Tab to see available models and functions
# nn.


a) Define the layers of the network in the `__init__` function and   
b) specify how data will pass through the network in the `forward` function.    
What is the [forward function?](https://stackoverflow.com/a/64989409) 

In [9]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # this is the place where you instantiate all your modules
        # you can later access them using the same names you've given them in here
        
        self.flatten = nn.Flatten()  # Covert the 28x28 image features to a "single row" 784 vector.
        self.linear_relu_stack = nn.Sequential(  # Sequential model
            # First hidden layer.
            nn.Linear(in_features=28*28, out_features=512),  # The first parameters depends on the shape X of data.
            nn.ReLU(),
            # Second Hidden layer
            nn.Linear(512, 712),  # We should be able to multiply input layers and output layers. All are 512 in the tutorial.
            nn.ReLU(),
            # Third Hidden layer
            nn.Linear(712, 812),  # We should be able to multiply input layers and output layers. All are 512 in the tutorial.
            nn.ReLU(),
            # Output layer
            nn.Linear(in_features=812,out_features=10),  # Last parameter = len(classes)
            # there is no activation function input because it's handled by the loss function.
           
        )

    # it's the forward function that defines the network structure
    # we're accepting only a single input in here, but you can use 
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)  # Name defined above in the __init__ fucntion.
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=712, bias=True)
    (3): ReLU()
    (4): Linear(in_features=712, out_features=812, bias=True)
    (5): ReLU()
    (6): Linear(in_features=812, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).


------------------------------------------------------------------------


In [10]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=712, bias=True)
    (3): ReLU()
    (4): Linear(in_features=712, out_features=812, bias=True)
    (5): ReLU()
    (6): Linear(in_features=812, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0216, -0.0135,  0.0288,  ..., -0.0125,  0.0176,  0.0134],
        [ 0.0264,  0.0098, -0.0124,  ...,  0.0062,  0.0355,  0.0236]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0062, -0.0215], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([712, 512]) | Values : tensor([[-0.0254,  0.0080, -0.0239,  ...,  0.0261,  0.0293,  0.0186],
        [-0.0041,  0.0065,  0.0323,  ..., -0.0409, -0.0186,  0

### Hyperparameters

We define the following hyperparameters for training:

        Number of Epochs - the number times to iterate over the dataset

        Batch Size - the number of data samples propagated through the network before the parameters are updated

        Learning Rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.



In [11]:
# learning_rate = 1e-3
# batch_size = 64
# epochs = 5

Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [12]:
loss_fn = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop:   
* the model makes predictions on the training dataset (fed to it in batches), and
* backpropagates the prediction error to adjust the model\'s parameters.

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.


In [15]:
def test(dataloader, model, loss_fn):  # No optimizer parameter in test function.
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():  # No gradient calculation. Used for inference on testing.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.


In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302898  [   64/60000]
loss: 2.299575  [ 6464/60000]
loss: 2.296325  [12864/60000]
loss: 2.297765  [19264/60000]
loss: 2.283401  [25664/60000]
loss: 2.278083  [32064/60000]
loss: 2.279359  [38464/60000]
loss: 2.269799  [44864/60000]
loss: 2.276451  [51264/60000]
loss: 2.268627  [57664/60000]
Test Error: 
 Accuracy: 22.8%, Avg loss: 2.265667 

Epoch 2
-------------------------------
loss: 2.268036  [   64/60000]
loss: 2.268210  [ 6464/60000]
loss: 2.254630  [12864/60000]
loss: 2.264193  [19264/60000]
loss: 2.244182  [25664/60000]
loss: 2.225687  [32064/60000]
loss: 2.242065  [38464/60000]
loss: 2.219469  [44864/60000]
loss: 2.227043  [51264/60000]
loss: 2.211099  [57664/60000]
Test Error: 
 Accuracy: 34.6%, Avg loss: 2.207000 

Epoch 3
-------------------------------
loss: 2.212807  [   64/60000]
loss: 2.213654  [ 6464/60000]
loss: 2.180659  [12864/60000]
loss: 2.201034  [19264/60000]
loss: 2.165376  [25664/60000]
loss: 2.122384  [32064/600

Read more about [Training your model](optimization_tutorial.html).


------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [18]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.


In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
len(classes)

10

In [20]:
model.eval()
x, y = test_data[0][0], test_data[0][1]

In [21]:
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).



### Model explainability
[Tutorial](https://pytorch.org/tutorials/beginner/introyt/captumyt.html)

## Kaggle notebook 
https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy

In [22]:
from torch.autograd import Variable

In [23]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [26]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [27]:
model = FashionCNN()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [28]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295329  [   64/60000]
loss: 0.476211  [ 6464/60000]
loss: 0.371127  [12864/60000]
loss: 0.456604  [19264/60000]
loss: 0.545811  [25664/60000]
loss: 0.452465  [32064/60000]
loss: 0.273926  [38464/60000]
loss: 0.449083  [44864/60000]
loss: 0.396580  [51264/60000]
loss: 0.306472  [57664/60000]
Test Error: 
 Accuracy: 86.6%, Avg loss: 0.371582 

Epoch 2
-------------------------------
loss: 0.191975  [   64/60000]
loss: 0.320662  [ 6464/60000]
loss: 0.167816  [12864/60000]
loss: 0.388564  [19264/60000]
loss: 0.365874  [25664/60000]
loss: 0.470826  [32064/60000]
loss: 0.248557  [38464/60000]
loss: 0.384953  [44864/60000]
loss: 0.298662  [51264/60000]
loss: 0.263779  [57664/60000]
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.308892 

Epoch 3
-------------------------------
loss: 0.172958  [   64/60000]
loss: 0.280543  [ 6464/60000]
loss: 0.125427  [12864/60000]
loss: 0.258149  [19264/60000]
loss: 0.329505  [25664/60000]
loss: 0.403258  [32064/600

CIFAR tutorial

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

Time series tutorial
https://colab.research.google.com/github/jinglescode/time-series-forecasting-pytorch/blob/main/demo-predicting-stock-prices.ipynb